# Ensemble


Ensemble learning helps improve machine learning results by combining several models. This approach allows the production of better predictive performance compared to a single model.

Most ensemble methods use a single base learning algorithm i.e. learners of the same type, leading to homogeneous ensembles.

There are also some methods that use heterogeneous learners, i.e. learners of different types, leading to heterogeneous ensembles. In order for ensemble methods to be more accurate than any of its individual members, the base learners have to be as accurate as possible and as diverse as possible.



### Bagging

Bagging stands for bootstrap aggregation. One way to reduce the variance of an estimate is to average together multiple estimates. For example, we can train M different trees on different subsets of the data (chosen randomly with replacement).

Bagging uses bootstrap sampling to obtain the data subsets for training the base learners. For aggregating the outputs of base learners, bagging uses voting for classification and averaging for regression.

### Boosting

Boosting is a general ensemble method that creates a strong classifier from a number of weak classifiers.

This is done by building a model from the training data, then creating a second model that attempts to correct the errors from the first model. Models are added until the training set is predicted perfectly or a maximum number of models are added.

AdaBoost is one of the most successful boosting algorithms developed for binary classification.

### Libraries useful in Ensemble are listed below

### Import all the libraries required

In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

### Load the "letter-recognition" data

In [3]:
# import dataset
df = pd.read_csv("letter-recognition.data.txt", header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


### Split the dataset into training and testing parts (70-30 ratio with a random state value 30)

In [4]:
# Select the independent variables and the target attribute
X = df[df.columns[1:]] # Selecting the independent variables
Y = df[df.columns[0]] # selecting only the target lableled column
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [5]:
# Divide the dataset into training and testing partition
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state = 30)

### Q1. Ensemble Method by manipulation of Dataset (Bagged Decision Trees)

Bagging performs best with algorithms that have high variance. A popular example are decision trees, often constructed without pruning.

We will create decision tree classifiers with and without bagging ensemble method and compare their performance.

In [6]:
# Implement the decision tree classifier using entropy and random state value as 30
dtree_entropy = DecisionTreeClassifier(criterion='entropy', random_state = 30) 

In [7]:
# Use k-fold cross validation with k=5
dtree_entropy = dtree_entropy.fit(X_train,Y_train)
scores = cross_val_score(dtree_entropy, X_train, Y_train, cv=5, scoring='accuracy')
print('scores: ', scores)
print('mean score: ', scores.mean())

scores:  [0.865      0.86964286 0.85214286 0.86       0.87      ]
mean score:  0.8633571428571427


### Prediction and Evaluation

In [8]:
# Predict results on the testing part
predictions = dtree_entropy.predict(X_test)
print(predictions)

['A' 'L' 'O' ... 'N' 'T' 'C']


In [1]:
# Calculate and print confusion matrix and other performance measures 
#print(classification_report(Y_test,predictions))
#print("Confusion Matrix")
#print(confusion_matrix(Y_test,predictions))
#print("\n Accuracy")
#print(accuracy_score(Y_test,predictions))

### Comparison with Bagged Decision Tree

In [10]:
# Create a model using bagging using 5 decision tree classifiers
from sklearn.ensemble import BaggingClassifier

seed = 30
dtree = DecisionTreeClassifier(criterion='entropy', random_state = 30) 
num_trees = 5
dtree_bagging = BaggingClassifier(base_estimator=dtree, n_estimators=num_trees, random_state=seed)

In [11]:
# Use k-fold cross validation with k=5
dtree_bagging = dtree_bagging.fit(X_train,Y_train)
scores = cross_val_score(dtree_bagging, X_train, Y_train, cv=5, scoring='accuracy')
print('scores: ', scores)
print('mean score: ', scores.mean())

scores:  [0.88714286 0.89035714 0.89       0.88285714 0.88714286]
mean score:  0.8875


### Prediction and Evaluation

In [12]:
# Predict results on the testing part
predictions = dtree_bagging.predict(X_test)
print(predictions)

['A' 'L' 'O' ... 'N' 'T' 'C']


In [2]:
# Calculate and print confusion matrix and other performance measures 
#print(classification_report(Y_test,predictions))
#print("Confusion Matrix")
#print(confusion_matrix(Y_test,predictions))
#print("\n Accuracy")
#print(accuracy_score(Y_test,predictions))

### Q2. Ensemble Method by manipulation of Classifiers (using Voting Classifier)

The VotingClassifier takes in a list of different estimators as arguments and a voting method. The **hard** voting method uses the predicted labels and a majority rules system, while the **soft** voting method predicts a label based on the argmax/largest predicted value of the sum of the predicted probabilities.

After we provide the desired classifiers, we need to fit the resulting ensemble classifier object. We can then get predictions and use accuracy metrics.

In [14]:
#Import required library
from sklearn.ensemble import VotingClassifier

In [15]:
# Implement the different classifiers
c1 = DecisionTreeClassifier(criterion='entropy', random_state = 30) # decision tree
c2 = KNeighborsClassifier(n_neighbors=3, metric='euclidean') # 3NN
c3 = KNeighborsClassifier(n_neighbors=5, metric='euclidean') # 5NN euclidean
c4 = KNeighborsClassifier(n_neighbors=5, metric='manhattan') # 5NN manhattan
c5 = GaussianNB() # gaussian Naive Bayes

In [16]:
# Build Voting Classifier using above estimators and hard voting method
# Function to be used: VotingClassifier(estimators,voting)
# Estimators represent the base classifiers used taken as ('base classifier name', variable_name)
estimator = [("Decision_Tree", c1),
             ("3NN_Euclidean", c2),
             ("5NN_Euclidean", c3),
             ("5NN_Manhattan", c4),
             ("Naive_Bayes", c5)
            ]
vote_hard = VotingClassifier(estimators = estimator, voting ='hard')

In [17]:
# Fit the voting classifier model and print scores using k-fold cross validation with k=5
vote_hard = vote_hard.fit(X_train,Y_train)
scores = cross_val_score(vote_hard, X_train, Y_train, cv=5, scoring='accuracy')
print('scores: ', scores)
print('mean score: ', scores.mean())

scores:  [0.94392857 0.9425     0.9425     0.945      0.94428571]
mean score:  0.9436428571428571


### Prediction and Evaluation

In [18]:
# Predict results on the testing part
predictions = vote_hard.predict(X_test)
print(predictions)

['A' 'L' 'O' ... 'N' 'T' 'C']


In [3]:
# Calculate and print confusion matrix and other performance measures 
#print(classification_report(Y_test,predictions))
#print("Confusion Matrix")
#print(confusion_matrix(Y_test,predictions))
#print("\n Accuracy")
#print(accuracy_score(Y_test,predictions))

### Q3. Manipulating the features

In [28]:
# Generate five random vectors
X1 = X.sample(10, axis = 1)
X2 = X.sample(10, axis = 1)
X3 = X.sample(10, axis = 1)
X4 = X.sample(10, axis = 1)
X5 = X.sample(10, axis = 1)

In [29]:
# Model 1
# Select the independent variables
# select only the target lableled column
# Train the model
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y, test_size=0.30, random_state = 30)
dtree1 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree1 = dtree1.fit(X_train1,Y_train1)

In [30]:
# Model 2
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y, test_size=0.30, random_state = 30)
dtree2 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree2 = dtree2.fit(X_train2,Y_train2)

In [31]:
# Model 3
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3, Y, test_size=0.30, random_state = 30)
dtree3 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree3 = dtree3.fit(X_train3,Y_train3)

In [32]:
# Model 4
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4, Y, test_size=0.30, random_state = 30)
dtree4 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree4 = dtree4.fit(X_train4,Y_train4)

In [33]:
# Model 5
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X5, Y, test_size=0.30, random_state = 30)
dtree5 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree5 = dtree5.fit(X_train5,Y_train5)

In [34]:
# Apply Voting Classifier
import numpy as np
from scipy import stats
predictions = np.vstack((dtree1.predict(X_test1), dtree2.predict(X_test2),
                        dtree3.predict(X_test3), dtree4.predict(X_test4), dtree5.predict(X_test5)))
predictions = stats.mode(predictions)[0].flatten()

In [5]:
# Calculate and print confusion matrix and other performance measures 
#print(classification_report(Y_test1,predictions)) # Y_test same as Y_test1
#print("Confusion Matrix")
#print(confusion_matrix(Y_test1,predictions))
#print("\n Accuracy")
#print(accuracy_score(Y_test1,predictions))

### Q4. Manipulating the classes

In [53]:
# Generate 5 sets of two class representation
import random
def initializer(Set, x):
    if x in Set:
        return 1
    else:
        return 0

alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
             'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
set1 = random.sample(alphabets, 10)
set2 = random.sample(alphabets, 10)
set3 = random.sample(alphabets, 10)
set4 = random.sample(alphabets, 10)
set5 = random.sample(alphabets, 10)

Y1 = np.array([initializer(set1, y) for y in Y])
Y2 = np.array([initializer(set2, y) for y in Y])
Y3 = np.array([initializer(set3, y) for y in Y])
Y4 = np.array([initializer(set4, y) for y in Y])
Y5 = np.array([initializer(set5, y) for y in Y])

In [47]:
# Model 1
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, Y1, test_size=0.30, random_state = 30)
dtree1 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree1 = dtree1.fit(X_train1,Y_train1)

In [48]:
# Model 2
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X, Y2, test_size=0.30, random_state = 30)
dtree2 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree2 = dtree2.fit(X_train2,Y_train2)

In [79]:
# Model 3
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X, Y3, test_size=0.30, random_state = 30)
dtree3 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree3 = dtree3.fit(X_train3,Y_train3)

In [49]:
# Model 4
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X, Y4, test_size=0.30, random_state = 30)
dtree4 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree4 = dtree4.fit(X_train4,Y_train4)

In [50]:
# Model 5
# Select the independent variables 
# select only the target lableled column
# Train the model
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X, Y5, test_size=0.30, random_state = 30)
dtree5 = DecisionTreeClassifier(criterion='entropy', random_state = 30)
dtree5 = dtree5.fit(X_train5,Y_train5)

In [84]:
# Apply Voting Classifier
import numpy as np
from scipy import stats
p1 = dtree1.predict(X_test1).tolist()
p2 = dtree2.predict(X_test2).tolist()
p3 = dtree3.predict(X_test3).tolist()
p4 = dtree4.predict(X_test4).tolist()
p5 = dtree5.predict(X_test5).tolist()
predictions = []

for i in range(len(p1)):
    alphavotes = [0 for i in range(26)]
    for j in range(26):
        if alphabets[j] in set1:
            alphavotes[j] += p1[i]
        if alphabets[j] in set2:
            alphavotes[j] += p2[i]
        if alphabets[j] in set3:
            alphavotes[j] += p3[i]
        if alphabets[j] in set4:
            alphavotes[j] += p4[i]
        if alphabets[j] in set5:
            alphavotes[j] += p5[i]
    predictions.append(alphabets[alphavotes.index(max(alphavotes))])
predictions = np.array(predictions)

['V' 'A' 'A' ... 'A' 'A' 'B']


In [4]:
# Calculate and print confusion matrix and other performance measures
#print(classification_report(Y_test,predictions))
#print("Confusion Matrix")
#print(confusion_matrix(Y_test,predictions))
#print("\n Accuracy")
#print(accuracy_score(Y_test,predictions))

### Q5. Which method performs the best

In [86]:
# Manipulation of Classifiers (Q2) works best